# Parametric non-intrusive model order reduction for 2D wildland fire model (without wind)

Here we describe the application of methods sPOD-NN, sPOD-I and POD-NN on a 2D wildland fire model without any topological changes. The model equations are given in Eq.(30) and Eq.(31) of the paper. The wind velocity is considered to be (0, 0). The results generated in this script are shown in Sec.(3.1.2) of the paper as well.

In [ ]:
import sys
sys.path.append('../sPOD/lib/')
sys.path.append('../DL-ROM/LIB/')

In [ ]:
from wildfire2D_sup import wildfire2D_sup
import numpy as np

## Basis reconstruction for the 2D wildland fire model

In this part we : 

<ul>
<li>Load the 2D wildland fire data.</li>
<li>Perform sPOD and POD on the generated data.</li>
<li>Extract the time amplitudes according to Eq.(8) and Eq.(13) from the paper.</li>
</ul>

The inputs here include : 
<ul>
<li>$variable$ which is to be set as 0 for "Temperature" and 1 for "Supply mass fraction".</li>
<li>$test\_val$ is the parameter value at which the testing is performed.</li>
</ul>

The sPOD algorithm has to run only once for the basis reconstruction and the results necessary for subsequent steps are stored to save repeated time and effort.

In [ ]:
variable = 0
test_val = 558.49

if variable == 0:
    name = "T"
else:
    name = "S"

In [ ]:
import os

q = np.load(os.path.abspath(".") + '/wildfire_data/2D/' + 'SnapShotMatrix' + str(test_val) + '.npy')
shifts_test = np.load(os.path.abspath(".") + '/wildfire_data/2D/' + 'Shifts' + str(test_val) + '.npy')

df = wildfire2D_sup(q, shifts_test, param_test_val=test_val, var=variable)

sPOD to be performed only once and the results to be stored

In [ ]:
############################# Run shifted POD on the data ########################## (only once)
impath = "./wildfire_data/2D/save_Wildfire/" + name + "/"
import os
import pickle
os.makedirs(impath, exist_ok=True)

U_list, TA_list_training, TA_list_interp, spod_modes = df.run_sPOD(spod_iter=2000)

with open(impath + 'U_list.data', 'wb') as filehandle:
    pickle.dump(U_list, filehandle)
with open(impath + 'TA_list_training.data', 'wb') as filehandle:
    pickle.dump(TA_list_training, filehandle)
with open(impath + 'TA_list_interp.data', 'wb') as filehandle:
    pickle.dump(TA_list_interp, filehandle)
with open(impath + 'spod_modes.data', 'wb') as filehandle:
    pickle.dump(spod_modes, filehandle)
with open(impath + 'Q_polar_train.data', 'wb') as filehandle:
    pickle.dump(df.q_polar_train, filehandle)

In [ ]:
impath = "./wildfire_data/2D/save_Wildfire/" + name + "/"
import os
import pickle

with open(impath + 'U_list.data', 'rb') as filehandle:
    U_list = pickle.load(filehandle) 
with open(impath + 'TA_list_training.data', 'rb') as filehandle:
    TA_list_training = pickle.load(filehandle)  
with open(impath + 'TA_list_interp.data', 'rb') as filehandle:
    TA_list_interp = pickle.load(filehandle) 
with open(impath + 'spod_modes.data', 'rb') as filehandle:
    spod_modes = pickle.load(filehandle) 
with open(impath + 'Q_polar_train.data', 'rb') as filehandle:
    Q_polar_train = pickle.load(filehandle) 
    
TA_TRAIN = np.concatenate(TA_list_training, axis=0)
SHIFTS_TRAIN = df.shifts_train[0][0]
PARAMS_TRAIN = df.params_train

POD

In [ ]:
from sklearn.utils.extmath import randomized_svd

u, s, vt = randomized_svd(np.concatenate(df.q_train, axis=1), n_components=sum(spod_modes) + 1, random_state=None)
U_POD_TRAIN = u
TA_POD_TRAIN = np.diag(s) @ vt

Here the testing data is assembled. The sPOD and POD are performed on the testing data and then time amplitudes and shifts are extracted. These are used for final error calculations. The sPOD on the testing data is also performed only once for saving time and effort.

In [ ]:
############################# Run shifted POD on the test data ########################## (only once)
import os
impath = "./wildfire_data/2D/save_Wildfire/" + name + "/" + str(test_val) + "/"
import pickle
os.makedirs(impath, exist_ok=True)

Q_frames_test_polar, Q_frames_test_cart, conv_param = df.test_data(spod_iter=1000)


with open(impath + 'Q_frames_test_polar.data', 'wb') as filehandle:
    pickle.dump(Q_frames_test_polar, filehandle)
with open(impath + 'Q_frames_test_cart.data', 'wb') as filehandle:
    pickle.dump(Q_frames_test_cart, filehandle)
with open(impath + 'Q_test_polar.data', 'wb') as filehandle:
    pickle.dump(df.q_polar_test, filehandle)
with open(impath + 'conv_param.data', 'wb') as filehandle:
    pickle.dump(conv_param, filehandle)

In [ ]:
impath = "./wildfire_data/2D/save_Wildfire/" + name + "/" + str(test_val) + "/"
import os
import pickle

with open(impath + 'Q_frames_test_polar.data', 'rb') as filehandle:
    Q_frames_test_polar = pickle.load(filehandle) 
with open(impath + 'Q_frames_test_cart.data', 'rb') as filehandle:
    Q_frames_test_cart = pickle.load(filehandle) 
with open(impath + 'Q_test_polar.data', 'rb') as filehandle:
    Q_test_polar = pickle.load(filehandle) 
with open(impath + 'conv_param.data', 'rb') as filehandle:
    conv_param = pickle.load(filehandle)
    
# Plot the frames for test parameter
df.plot_sPOD_frames(Q_frames_test_cart, plot_every=10, var_name="T")

In [ ]:
mu_vecs_test = np.asarray([df.param_test_val])
params_test = [np.squeeze(np.asarray([[np.ones_like(df.t) * mu], [df.t]])) for mu in mu_vecs_test]
PARAMS_TEST = np.concatenate(params_test, axis=1)

q1_test = Q_frames_test_polar[0]
q2_test = Q_frames_test_polar[1]
time_amplitudes_1_test = U_list[0].transpose() @ q1_test
time_amplitudes_2_test = U_list[1].transpose() @ q2_test

TA_TEST = np.concatenate((time_amplitudes_1_test, time_amplitudes_2_test), axis=0)
SHIFTS_TEST = df.shifts_test[0][0]
TA_POD_TEST = U_POD_TRAIN.transpose() @ df.q_test

We assemble the $\hat{A}$ matrix according to the Eq.(16) from the paper.

In [ ]:
shifts_train = np.reshape(SHIFTS_TRAIN, newshape=[1, -1])
shifts_test = np.reshape(SHIFTS_TEST, newshape=[1, -1])

ta_train = np.concatenate((TA_TRAIN, shifts_train), axis=0)
ta_test = np.concatenate((TA_TEST, shifts_test), axis=0)

In [ ]:
print("Grid, Nx : {}, Ny : {}, Nt : {}".format(len(df.x), len(df.y), len(df.t)))
print("Number of sPOD frames : {}".format(len(spod_modes)))
print("Number of modes (frame wise) : {}, {}".format(spod_modes[0], spod_modes[1]))
print("Size of training matrix : {} x {}".format(int(ta_train.shape[0]), int(ta_train.shape[1])))

## Neural network training

Based on the data which we obtain from the previous step we train our neural network. For the training we first define certain parameters needed for training step. The parameters are mentioned here are:

<ul>
<li>$scaling$ activates the min-max data scaling for efficient training.</li>
<li>$full\_order\_model\_dimension$ is effectively $M$ which is the total number of grid points.</li>
<li>$reduced\_order\_model\_dimension$ is $n_{\mathrm{dof}}$ mentioned in Eq.(19) in the paper.</li>
<li>$totalModes$ is the total number of modes.</li>
<li>$num\_early\_stop$ defines the early stopping criteria for training step.</li>
</ul>

Subsequently the hyperparameters are:
<ul>
<li>$epochs$ sets the total number of epochs for training.</li>
<li>$lr$ sets the learning rate for training.</li>
<li>$loss\_type$ is the type of loss to consider while training options being $L1$ or $MSE$.</li>
<li>$batch\_size$ sets the total number of minibatches for the training data to be broken down into for effective training.</li>
</ul>

In [ ]:
params_sPOD = {
        'scaling': True,
        'full_order_model_dimension': len(df.x) * len(df.y), 
        'reduced_order_model_dimension': ta_train.shape[0],
        'totalModes': ta_train.shape[0] - len(spod_modes) + 1, 
        'num_early_stop': 4000 
    }
params_POD = {
        'scaling': True, 
        'full_order_model_dimension': len(df.x) * len(df.y), 
        'reduced_order_model_dimension': TA_POD_TRAIN.shape[0], 
        'totalModes': TA_POD_TRAIN.shape[0], 
        'num_early_stop': 4000 
    }

In [ ]:
# training the model
from DFNN import run_model 
import time
tic_sPOD = time.process_time() 
print("#################################")
print("sPOD-NN")
model_sPOD, _, scaling_sPOD = run_model(ta_train, PARAMS_TRAIN, epochs=200000, lr=0.005, loss_type='L1', 
                                        logs_folder='./DNN_result/wildfire2D/training_results_sPOD/' + name, 
                                        params=params_sPOD, batch_size=50)
print("#################################\n")
toc_sPOD = time.process_time()

tic_POD = time.process_time()
print("#################################")
print("POD-NN")
model_POD, _, scaling_POD = run_model(TA_POD_TRAIN, PARAMS_TRAIN, epochs=200000, lr=0.005, loss_type='L1', 
                                      logs_folder='./DNN_result/wildfire2D/training_results_POD/' + name, 
                                      params=params_POD, batch_size=50)
print("#################################\n")
toc_POD = time.process_time()

print(f"Time consumption in training (sPOD-NN) : {toc_sPOD - tic_sPOD:0.4f} seconds")
print(f"Time consumption in training (POD-NN) : {toc_POD - tic_POD:0.4f} seconds")

## Neural network prediction

After the training is finished the best weights are saved for network prediction. Here those weights are loaded and the prediction is performed. The dictionary $test$ is defined here which determines whether to run a multi-query scenario or full prediction scenario. If $test['typeOfTest'] = "query"$ then the multi-query scenario is run for which $test['typeOfTest'] = 40$ sets the time step at which the prediction has to be performed. 

Plotting function is only activated for $test['typeOfTest'] = "full"$ which gives us the full prediction throughout all the time steps.

In [ ]:
test = {
    'typeOfTest': "full",
    'test_sample': 40
}

In [ ]:
import torch
import pathlib
import os
from DFNN import scale_params

# Load the correct model
log_folder_base_sPOD = 'DNN_result/wildfire2D/training_results_sPOD/' + name + '/'
log_folder_trained_model_sPOD = sorted(pathlib.Path(log_folder_base_sPOD).glob('*/'), key=os.path.getmtime)[-1]
PATH_sPOD = str(log_folder_trained_model_sPOD) + '/trained_weights/' + 'weights.pt'

log_folder_base_POD = 'DNN_result/wildfire2D/training_results_POD/' + name + '/'
log_folder_trained_model_POD = sorted(pathlib.Path(log_folder_base_POD).glob('*/'), key=os.path.getmtime)[-1]
PATH_POD = str(log_folder_trained_model_POD) + '/trained_weights/' + 'weights.pt'

PATH_sPOD = 'DNN_result/wildfire2D/training_results_sPOD/' + name + '/2023_03_09__10-35-48/trained_weights/weights.pt'
PATH_POD = 'DNN_result/wildfire2D/training_results_POD/' + name + '/2023_03_09__10-44-59/trained_weights/weights.pt'

# Scale the parameters before prediction
if '/trained_weights/weights.pt' in PATH_sPOD: address_sPOD = PATH_sPOD.replace('/trained_weights/weights.pt', '')
scaling_sPOD = np.load(address_sPOD + '/variables/' + 'scaling.npy', allow_pickle=True)

if '/trained_weights/weights.pt' in PATH_POD: address_POD = PATH_POD.replace('/trained_weights/weights.pt', '')
scaling_POD = np.load(address_POD + '/variables/' + 'scaling.npy', allow_pickle=True)

PARAMS_TEST_sPOD = scale_params(PARAMS_TEST, params_sPOD, scaling_sPOD)
PARAMS_TEST_POD = scale_params(PARAMS_TEST, params_POD, scaling_POD)

In [ ]:
if test['typeOfTest'] == "query":
    test_sample = test['test_sample']
    
    ta_test = ta_test[:, test_sample][..., np.newaxis]
    
    TA_TEST = TA_TEST[:, test_sample][..., np.newaxis]
    TA_POD_TEST = TA_POD_TEST[:, test_sample][..., np.newaxis]
    
    tmp = []
    for i in range(df.NumFrames):
        tt = []
        for m in range(spod_modes[i]):
            ampl = TA_list_interp[i][m][test_sample, :][np.newaxis, ...]
            tt.append(ampl)
        tmp.append(tt)
    TA_list_interp = tmp
    
    SHIFTS_TEST = SHIFTS_TEST[test_sample]
    
    PARAMS_TEST_sPOD = PARAMS_TEST_sPOD[:, test_sample][..., np.newaxis]
    PARAMS_TEST_POD = PARAMS_TEST_POD[:, test_sample][..., np.newaxis]

In [ ]:
# testing the model
from DFNN import test_model 
import time 

tic = time.process_time()
rel_err_sPOD, results_predicted_sPOD = test_model(ta_test, PARAMS_TEST_sPOD, trained_model=None, saved_model=True, 
                                                  PATH_TO_WEIGHTS=PATH_sPOD, params=params_sPOD, scaling=scaling_sPOD, 
                                                  batch_size=50) 
toc = time.process_time()
print(f"Time consumption in testing sPOD-NN model : {toc - tic:0.4f} seconds")

tic = time.process_time()
rel_err_POD, results_predicted_POD = test_model(TA_POD_TEST, PARAMS_TEST_POD, trained_model=None, saved_model=True, 
                                                PATH_TO_WEIGHTS=PATH_POD, params=params_POD, scaling=scaling_POD, 
                                                batch_size=50)
toc = time.process_time()
print(f"Time consumption in testing POD-NN model : {toc - tic:0.4f} seconds")

Once the predictions for the time amplitudes and the shifts had been made we now reconstruct the snapshot according to Eq.(11) and Eq.(18) for POD and sPOD based methods respectively

In [ ]:
frame_amplitudes_predicted_sPOD = results_predicted_sPOD[:-1, :]
shifts_predicted_sPOD = results_predicted_sPOD[-1:, :]
frame_amplitudes_predicted_POD = results_predicted_POD

In [ ]:
Q_recon_sPOD_cart, Q_recon_POD_cart, Q_recon_interp_cart, errors = df.plot_online_data(frame_amplitudes_predicted_sPOD, 
                                                                                       frame_amplitudes_predicted_POD, 
                                                                                       TA_TEST, TA_POD_TEST, TA_list_interp,
                                                                                       shifts_predicted_sPOD, SHIFTS_TEST, 
                                                                                       spod_modes, U_list, U_POD_TRAIN, 
                                                                                       Q_test_polar, Q_frames_test_polar,
                                                                                       conv_param, plot_online=True, 
                                                                                       test_type=test)

In [ ]:
if test['typeOfTest'] != "query":
    df.plot_recon(Q_recon_sPOD_cart, Q_recon_POD_cart, Q_recon_interp_cart, t_a=10, t_b=100)

## Reconstruction error plots 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from Helper import save_fig
from statistics import mean

impath = "../plots/images_wildfire2D/"
os.makedirs(impath, exist_ok=True) 

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern"]})

SMALL_SIZE = 16   # 16
MEDIUM_SIZE = 18   # 18
BIGGER_SIZE = 20   # 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
truncated_modes = np.array([2 + 1, 7 + 1, 9 + 1, 13 + 1, 20 + 1, 24 + 1])
E_sPOD_NN = np.array([0.20504, 0.02914, 0.02397, 0.02638, 0.02279, 0.02942])
E_sPOD_I = np.array([0.20452, 0.03269, 0.03014, 0.02903, 0.02867, 0.02865])
E_POD_NN = np.array([0.69972, 0.37205, 0.30384, 0.21423, 0.13170, 0.11663])

err = errors[0]
err_max = [max(x) for x in err]
err_min = [min(x) for x in err]
err_mean = [mean(x) for x in err]

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

axs[0].semilogy(truncated_modes, E_sPOD_NN, color="red", linestyle='--', marker="*", label=r"$E^{\mathrm{sPOD-NN}}_{\mathrm{tot}}$")
axs[0].semilogy(truncated_modes, E_sPOD_I, color="blue", linestyle='--', marker="*", label=r"$E^{\mathrm{sPOD-I}}_{\mathrm{tot}}$")
axs[0].semilogy(truncated_modes, E_POD_NN, color="black", linestyle='--', marker="*", label=r"$E^{\mathrm{POD-NN}}_{\mathrm{tot}}$")
axs[0].set_xlabel(r"$n_{\mathrm{dof}}$")
axs[0].set_ylabel('Errors')
axs[0].grid()
axs[0].legend(loc='upper right')

axs[1].semilogy(df.t, err_max, color="teal", linestyle='--', label=r"max$(E^{\mathrm{sPOD-NN}}_j)$")
axs[1].semilogy(df.t, err_mean, color="orange", linestyle='--', label=r"mean$(E^{\mathrm{sPOD-NN}}_j)$")
axs[1].semilogy(df.t, err_min, color="dimgrey", linestyle='--', label=r"min$(E^{\mathrm{sPOD-NN}}_j)$")
axs[1].set_xlabel(r"time $t$")
axs[1].grid()
axs[1].legend(loc='center right')

save_fig(filepath=impath + 'Rel_err_2D', figure=fig)
fig.savefig(impath + "Rel_err_2D" + ".pdf", format='pdf',dpi=200, transparent=True, bbox_inches="tight")